<a href="https://colab.research.google.com/github/Sicily-F/cagedbirdID/blob/main/10_Evaluating_model_performance_using_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10. Evaluating model performance using cross validation

If the performance of the single train-test split can be widely retained after repeated testing, this suggests that the model generalizes reasonably within the domain tested, which in this case was primarily wildlife markets. However, it is possible to accidentally train on a subset that does not reflect a real-world situation. The validation or test datasets could consist of easier examples than the training dataset. By training the model on randomly shuffled training, validation, and test datasets, any tendencies towards overfitting can be avoided.


Here we show an example of cross-validation. This can be done with any of your model, though we show a vision transformer model here. Essentially it's the same code as file 8., just in one giant for loop!

There are other ways to perform kfold validation using the [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) package, and ours is certainly not the fastest. However, given that we used the TensorFlow ImageDataGenerator, we had to use this version, so we could preserve our initial folder structure.


We wrote a *for loop* to split our intial data (a folder named 'new_all_species', which had a folder for each 37 species), into 5 different folders, each which represents a randomised complication of the original data with 70% for training, 15% for validation and 15% for testing. We used the amazing [splitfolders](https://pypi.org/project/split-folders/) package to do this!


In [ ]:
import splitfolders

# This runs the loop 5 times
m = 5
for i in range(m):
    input_folder = 'F:/new_all_species/'
    output = 'F:/refolds/'+ str(i)
    splitfolders.ratio(input_folder, output=output, seed=None, ratio=(.7, .15, .15), group_prefix=None) 


  # This results in 5 folders named 0,1,2,3,4


In [ ]:
import numpy as np	
import pandas as pd
from pandas import DataFrame
import time
import PIL.Image as Image
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_addons as tfa
import PIL
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

RANDOM_SEED = 1
IMAGE_SIZE = 224
BATCH_SIZE = 16

import numpy as np
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=True): # was False before, as true still doesn't help me do the pixel level implementation 
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser
	
root = 'refolds/'

os.walk(root)

for folder in os.listdir(root):
    datagen = ImageDataGenerator(
    rescale=1/255,
    preprocessing_function=get_random_eraser(v_l=0, v_h=255, pixel_level=True),
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
    

    train_gen = datagen.flow_from_directory(
       root + folder + '/train', 
        target_size=(IMAGE_SIZE , IMAGE_SIZE), 
        batch_size=BATCH_SIZE,
        seed=RANDOM_SEED
    )

    testgen = ImageDataGenerator(
        rescale=1/255)
        
    val_gen = testgen.flow_from_directory(
        root + folder + '/val', 
        target_size=(IMAGE_SIZE, IMAGE_SIZE), 
        batch_size=BATCH_SIZE,
        seed=RANDOM_SEED
    )
    
    test_gen = testgen.flow_from_directory(
        root + folder + '/test', 
        target_size=(IMAGE_SIZE, IMAGE_SIZE), 
        batch_size= BATCH_SIZE,
        shuffle=False,
        seed=RANDOM_SEED
    )
    
    # create the base pre-trained model
    from vit_keras import vit

    vit_model = vit.vit_b32(
            image_size = IMAGE_SIZE,
            activation = 'softmax',
            pretrained = True,
            include_top = False,
            pretrained_top = False,
            classes = 37)
    		    
    model = tf.keras.Sequential([
            vit_model,
            tf.keras.layers.Flatten(),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(11, activation = 'relu'), #was tfa.activations.gelu
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(37, 'softmax')
        ],
        name = 'vision_transformer')
    
    model.summary()
    
    # training the model
    learning_rate = 1e-4
    
    optimizer = Adam(learning_rate = learning_rate) 
    
    model.compile(optimizer = optimizer, 
                  loss = tf.keras.losses.CategoricalCrossentropy(), 
                  metrics = ['accuracy'])
    
    STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
    STEP_SIZE_VAL = val_gen.n // val_gen.batch_size
     
    
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                     min_delta = 1e-4,
                                                     patience = 3, 
                                                     mode = 'max',
                                                     restore_best_weights = True,
                                                     verbose = 1)
    
    
    filepath = "visontmodel"+str(folder)+".h5" 
    
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='min')
    
  
    callbacks_list = [earlystopping, checkpoint]
    
    EPOCHS = 100

    history = model.fit(train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VAL,
          epochs = EPOCHS,
          callbacks = callbacks_list)
    
        
    test_batch_x, test_batch_y = test_gen.next()
    pred_batch = model.predict(test_batch_x)
    
    test_labels = np.argmax(test_batch_y, axis=1)
    test_pred = np.argmax(pred_batch, axis=1)
    
    test_acc = sum(test_labels == test_pred) / len(test_labels)
    print('Accuracy str(folder): %.3f' % test_acc) 
     
    Y_pred = model.predict(test_gen, test_gen.samples // BATCH_SIZE+1)
    y_pred = np.argmax(Y_pred, axis=1)

    from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
    print(precision_score(test_gen.classes, y_pred , average="macro"))
    print(recall_score(test_gen.classes, y_pred , average="macro"))
    print(f1_score(test_gen.classes, y_pred , average="macro"))
    
    from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
    Y_pred = model.predict(test_gen, test_gen.samples // BATCH_SIZE+1)
    y_pred = np.argmax(Y_pred, axis=1)
    
    
    from sklearn.metrics import classification_report, confusion_matrix
    # code found here: https://stackoverflow.com/questions/66386561/keras-classification-report-accuracy-is-different-between-model-predict-accurac?noredirect=1&lq=1
    
    print(confusion_matrix(test_gen.classes, y_pred))
    
    print('Classification Report')
    
    target_names = ['asian_pied', 'bali_myna', 'bluethroat', 'bm_leafbird', 'bn_oriole', 'bw_leafbird', 'bw_myna', 'c_laughingthrush', 'c_whiteeye', 'cc_thrush', 'chestnut_munia', 
                    'common_myna', 'fischers_lovebird', 'gg_leafbird', 'grey_parrot', 'grosbeak', 'hill_myna', 'hwamei',
                    'javan_sparrow', 'jg_magpie', 'leiothrix', 'lt_shrike', 'oh_thrush', 'om_robin', 'r_laughingthrush', 
                    'red_billedstarling', 'red_whiskered', 'rubythroat', 'sb_munia', 'sh_bulbul', 'silver_eared',
                    'su_laughingthrush', 'swinhoes', 'wr_munia', 'ws_shama', 'z_dove', 'z_finch']
    
    print(classification_report(test_gen.classes, y_pred, target_names=target_names))
    
    cm = confusion_matrix(test_gen.classes, y_pred)
    
    import itertools
    
    def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Oranges):#was Blues before, refer here for help: https://stackoverflow.com/questions/57043260/how-change-the-color-of-boxes-in-confusion-matrix-using-sklearn
    
        """
    
        This function prints and plots the confusion matrix.
    
        Normalization can be applied by setting `normalize=True`.
    
        """
    
        plt.figure(figsize=(20,20))
    
    
    
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
        plt.title(title)
    
        plt.colorbar()
    
    
    
        tick_marks = np.arange(len(classes))
    
        plt.xticks(tick_marks, classes, rotation=90)
    
        plt.yticks(tick_marks, classes)
    
    
    
        if normalize:
    
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
            cm = np.around(cm, decimals=2)
    
            cm[np.isnan(cm)] = 0.0
    
            print("Normalized confusion matrix")
    
        else:
    
            print('Confusion matrix, without normalization')
    
        thresh = cm.max() / 2.
    
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    
            plt.text(j, i, cm[i, j],
    
                     horizontalalignment="center",
    
                     color="white" if cm[i, j] > thresh else "black")
    
        plt.tight_layout()
    
        plt.ylabel('True label')
    
        plt.xlabel('Predicted label')
        
    plot_confusion_matrix(cm, target_names, title='Confusion Matrix')
    
# This plots a confusion matrix for each fold